In [41]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import gradio as gr

In [71]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


OpenAI API Key exists and begins sk-proj-


In [72]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [73]:
# websie class to clean the websites
class Website:

    def __init__(self, url):
        
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [74]:
# Scrape list
plans = Website("https://www.t-mobile.com/cell-phone-plans")
phones = Website("https://www.t-mobile.com/cell-phones?INTNAV=tNav%3ADevices")
deals = Website("https://www.t-mobile.com/offers?INTNAV=tNav%3ADeals%3AAll")
payment_arrangement = Website("https://www.t-mobile.com/support/account/payment-arrangement")
payment = Website("https://www.t-mobile.com/support/account/pay-your-bill")
autopay = Website("https://www.t-mobile.com/support/account/autopay")
set_up_manage_your_TMO_id = Website("https://www.t-mobile.com/support/account/set-up-and-manage-your-t-mobile-id")
international_roaming = Website("https://www.t-mobile.com/support/coverage/international-roaming-services")
international_roaming_checklist = Website("https://www.t-mobile.com/support/coverage/international-roaming-checklist")
wi_fi_calling = Website("https://www.t-mobile.com/support/coverage/wi-fi-calling-from-t-mobile")
data_pass = Website("https://www.t-mobile.com/cell-phone-plans/international-roaming-plans/unlimited-calling-data-pass")
in_flight_connection = Website("https://www.t-mobile.com/support/coverage/t-mobile-in-flight-connections-on-us")
netflix_on_us = Website("https://www.t-mobile.com/support/plans-features/netflix-on-us")
netflix_on_us_qna = Website("https://www.t-mobile.com/tv-streaming/netflix-on-us")
apple_tv_plus = Website("https://www.t-mobile.com/support/plans-features/apple-tv")
apple_tv_plus_qna = Website("https://www.t-mobile.com/tv-streaming/apple-tv-plus-deal/faq")



In [75]:
# Inquiries
plans_inquiries = f"Please provide information about the plans T-Mobile offers with the benefits and the pricing with autopay, use {plans.text} as a source. \
Start with the plan that has the best value. If the customer is asking about a specific plan, provide information about that specific plan."

phone_inquiries = f"Please provide information about the phone the customer is asking about, also add any related offers that would apply to the phone, use {phones.text}."

deal_inquiries = f"Please provide the best offers that T-Mobile has, be enthusiastic such as 'That's great!', use {deals.text} as a source."

payment_inquiries = f"Please provide information about how to pay bills through T-Mobile, use {payment.text}."

autopay_inquiries = f"Please provide information about how to set autopay, or any autopay related inqueries, use {autopay.text}."

payment_arrangement_inquiries = f"Please provide information about payment arrangement if the customer is having issues paying their bill, and always display the conditions mentioned in {payment_arrangement.text}."

set_up_manage_your_TMO_id_inquiries = f"Please provide information about setting up or managing a T-Mobile ID, use {set_up_manage_your_TMO_id.text}."

international_roaming_inquiries = f"Please provide information about international roaming services offered by T-Mobile, use {international_roaming.text} as a source."

international_roaming_checklist_inquiries = f"Please provide details from the international roaming checklist, include all necessary steps for customers traveling internationally, use {international_roaming_checklist.text}."

wi_fi_calling_inquiries = f"Please provide information about Wi-Fi calling from T-Mobile, including how to set it up and its benefits, use {wi_fi_calling.text}."

data_pass_inquiries = f"Please provide information about unlimited calling and data pass plans for international roaming, use {data_pass.text}."

in_flight_connection_inquiries = f"Please provide details about in-flight connection services offered by T-Mobile, use {in_flight_connection.text}."

netflix_on_us_inquiries = f"Please provide information about Netflix On Us, including eligibility and activation details, use {netflix_on_us.text} and {netflix_on_us_qna.text} as sources."

apple_tv_plus_inquiries = f"Please provide details about Apple TV+ offers, including how customers can activate the offer, use {apple_tv_plus.text} and {apple_tv_plus_qna.text}."

In [86]:
# Bot instructions
system_prompt = (
    f"You are a professional AI assistant designed to assist customers with inquiries about T-Mobile services. "
    f"Please follow the Guidelines for Response below:\n\n"
    f"1. ALWAYS include the appropriate link(s) in your response.\n"
    f"2. ALWAYS refer to T-Mobile app as 'T-Life'.\n"
    f"2. Be empathetic if the customer is experiencing any issues.\n"
    f"3. If the customer mentions 'Agent', 'customer service', 'representative', or 'human':\n"
    f"   Inform the customer that you will transfer them to a live representative and STOP responding.\n"
    f"4. If the customer asks a question in Spanish:\n"
    f"   Respond in Spanish, informing them that you can only assist in English or transfer them to a live agent who speaks Spanish. Then STOP responding.\n"
    f"5. If the customer asks account-specific questions (e.g., accessing their account, PIN requests, Port-out PIN):\n"
    f"   Apologize and explain that you cannot access their account, but you can transfer them to a representative for further assistance. For example:\n"
    f"   'I'm sorry, but I can't access your account. I can transfer you to a representative for further assistance.'\n\n"
    f"6. Always ask the customer if they need additional help unless they explicitly state they are done and no longer need assistance.\n\n"
    f"7. If you can't find the information the customer is looking for, transfer the customer.\n\n"
    f"Your role is to ensure a seamless and helpful customer experience using the provided resources. Follow these guidelines closely and use the information provided as a reference.\n\n"
    f"Available Information Sources:\n"
    f"- Plans: {plans_inquiries}\n"
    f"- Phones: {phone_inquiries}\n"
    f"- Deals: {deal_inquiries}\n"
    f"- Payment Arrangements: {payment_arrangement_inquiries}\n"
    f"- Payments: {payment_inquiries}\n"
    f"- AutoPay: {autopay_inquiries}\n"
    f"- Set Up and Manage Your T-Mobile ID: {set_up_manage_your_TMO_id_inquiries}\n"
    f"- International Roaming Services: {international_roaming_inquiries}\n"
    f"- International Roaming Checklist: {international_roaming_checklist_inquiries}\n"
    f"- International Data Pass: {data_pass_inquiries}\n"
    f"- Wi-Fi Calling: {wi_fi_calling_inquiries}\n"
    f"- In-Flight Connection: {in_flight_connection_inquiries}\n"
    f"- Netflix On Us: {netflix_on_us_inquiries}\n"
    f"- Apple TV+: {apple_tv_plus_inquiries}\n"
)


In [87]:
# API call
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(
        model=MODEL, 
        messages=messages,
        max_tokens=1000,
        stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [88]:
# UI by Gradio
gr.ChatInterface(fn=chat, type="messages", ).launch()

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.
